# Client Scaling of OSTs

This notebook visualizes the throughput of a fixed number of OSTs as client count increases.

The specific data being plotted came from a set of IOR runs that used file-per-process and shared-file (non-collective) POSIX I/O. The actual runs were conducted on a single Lustre OST on a single Lustre OSS (Cray Sonexion 2000) using an increasing number of Knights Landing nodes.

In [ ]:
%matplotlib inline

In [ ]:
import glob
import matplotlib
matplotlib.rcParams['font.size'] = 18
import matplotlib.pyplot
import pandas

In [ ]:
import iorparse

In [ ]:
OUTPUT_FORMAT = 'pdf' # png, jpg, ... or None to not save

In [ ]:
# Load results
raw_results = {}
for pattern in 'shared', 'fpp':
    raw_results[pattern] = []
    for ior_output in glob.glob('results/%s_*.out' % pattern):
        raw_results[pattern] += iorparse.parse(open(ior_output, 'r'), access='write')

results = {}
for pattern, raw_result in raw_results.items():
    results[pattern] = pandas.DataFrame.from_dict(raw_result)
    results[pattern]['nodes'] = (results[pattern]['tasks'] / results[pattern]['ppn']).astype('int32')
    results[pattern]['bw_gibs'] = results[pattern]['bw_mibs'] / 1024.0

In [ ]:
fig, ax = matplotlib.pyplot.subplots(figsize=(8,6))
for dsetno, (pattern, label) in enumerate([('shared', 'Shared File'), ('fpp', 'File Per Process')]):
    grouped = results[pattern].groupby(['nodes'])
    errors = grouped.std()['bw_gibs']
    grouped.mean()['bw_gibs'].plot(ax=ax,
                                   label=label,
                                   yerr=2*errors,
                                   linewidth=4,
                                   color='C%d' % (dsetno + 2),
                                   capsize=10,
                                   capthick=2,
                                   elinewidth=1)

ax.set_xscale('log', basex=2)
ax.set_xticklabels([int(x) for x in ax.get_xticks()])
ax.grid()
ax.set_axisbelow(True)
ax.set_xlabel("Number of Client Nodes (16 processes/node)")
ax.set_ylabel("Bandwidth (GiB/s)")
ax.legend()

In [ ]:
if OUTPUT_FORMAT:
    ax.get_figure().savefig('client-scaling.%s' % OUTPUT_FORMAT, bbox_inches='tight')